<a href="https://colab.research.google.com/github/threegenie/studying_pytorch/blob/main/04_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> PyTorch로 시작하는 딥러닝 입문 - <nn.Module로 구현하는 로지스틱 회귀, 클래스로 파이토치 모델 구현하기>를 참고하여 직접 logistic regression model을 만들어보기(sklearn과 pytorch 비교)
- Kaggle Titanic 데이터를 사용하여 생존자 예측(https://www.kaggle.com/c/titanic/data)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# random seed 고정
torch.manual_seed(1)

### Data import

In [3]:
train = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/titanic/train.csv')
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/titanic/test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Scikit-Learn으로 Logistic Regression 모델 만들기

In [5]:
# train, val split
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, random_state=2)

In [6]:
target = 'Survived'
features = ['Pclass','SibSp','Parch','Fare'] #scaling하기 귀찮아서 numeric한 특성만 사용..

In [7]:
y_train = train[target]
x_train = train[features]

In [8]:
x_val = val[features]
y_val = val[target]

In [9]:
# logistic regression model import

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
print('검증세트 정확도', logistic.score(x_val, y_val))

검증세트 정확도 0.6860986547085202


In [11]:
pred = logistic.predict(x_val)
pred

array([1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0])

### Pytorch로 Logistic Regression 모델 만들기

In [14]:
#1차원으로 만들기 위해 타겟 원소 따로 빼서 numpy array로 만듦
y_train = [[x] for x in train['Survived']]
y_val = [[x] for x in val['Survived']]

In [15]:
# 타겟 numpy to tensor
y_train_ts = torch.Tensor(y_train)
y_val_ts = torch.Tensor(y_val)

In [16]:
# convert pandas dataframe to numpy array
trainX = torch.tensor(x_train.values)
valX = torch.tensor(x_val.values)

In [17]:
# numpy array to Torch tensor
X_train_ts = trainX.float()
X_val_ts = valX.float()

In [18]:
# long tensor to float tensor
X_train_ts = X_train_ts.float()
y_train_ts = y_train_ts.float()

In [19]:
# logistic regression model built
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [22]:
model = BinaryClassifier()

In [25]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 2000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(X_train_ts)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train_ts)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train_ts # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/2000 Cost: 35.536613 Accuracy 62.57%
Epoch   10/2000 Cost: 35.536331 Accuracy 62.57%
Epoch   20/2000 Cost: 35.536057 Accuracy 62.57%
Epoch   30/2000 Cost: 35.535782 Accuracy 62.57%
Epoch   40/2000 Cost: 35.535500 Accuracy 62.57%
Epoch   50/2000 Cost: 35.535233 Accuracy 62.57%
Epoch   60/2000 Cost: 35.534958 Accuracy 62.57%
Epoch   70/2000 Cost: 35.534691 Accuracy 62.57%
Epoch   80/2000 Cost: 35.534420 Accuracy 62.57%
Epoch   90/2000 Cost: 35.534157 Accuracy 62.57%
Epoch  100/2000 Cost: 35.533886 Accuracy 62.57%
Epoch  110/2000 Cost: 35.533623 Accuracy 62.57%
Epoch  120/2000 Cost: 35.533363 Accuracy 62.57%
Epoch  130/2000 Cost: 35.533100 Accuracy 62.57%
Epoch  140/2000 Cost: 35.532841 Accuracy 62.57%
Epoch  150/2000 Cost: 35.532581 Accuracy 62.57%
Epoch  160/2000 Cost: 35.532322 Accuracy 62.57%
Epoch  170/2000 Cost: 35.532066 Accuracy 62.57%
Epoch  180/2000 Cost: 35.531818 Accuracy 62.57%
Epoch  190/2000 Cost: 35.531563 Accuracy 62.57%
Epoch  200/2000 Cost: 35.531307 Accuracy